In [2]:
#%pip install ultralytics

In [31]:
import requests
import cv2
from ultralytics import YOLO
import numpy as np
from PIL import Image, ImageDraw
import random
import gradio as gr

def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 


# 모델 로드
model = YOLO('yolo3/yolov8n.pt')

#response = requests.get("https://cdn.pixabay.com/photo/2025/09/23/05/50/girl-9849748_1280.jpg")
#image_data = response.content
# NumPy 배열로 변환
#image_np = np.frombuffer(image_data, np.uint8)
# OpenCV를 사용하여 NumPy 배열을 이미지(OpenCV BGR 포맷)로 디코딩
#image_arr = cv2.imdecode(image_np, cv2.IMREAD_COLOR)

def detect_objects(image_arr) :
    image_arr = image_arr.copy()
    image_arr = cv2.cvtColor(image_arr, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image_arr)
    draw = ImageDraw.Draw(image)

    # 객체 감지 수행
    response = model(image_arr)

    # 단일 이미지를 처리했으므로, 결과 리스트의 첫 번째 요소(0번째 인덱스)를 가져옴
    result = response[0]

    # 클래스 이름 목록: 모델이 인식할 수 있는 객체의 이름 (예: 'person', 'car', 'dog')
    label_list = result.names
    # xyxy 포맷(xmin, ymin, xmax, ymax)으로, NumPy 배열로 변환하고 CPU 메모리로 이동합니다.
    bounding_box_list = result.boxes.xyxy.cpu().numpy()
    # 신뢰도 목록: 각 객체 감지에 대한 모델의 확신도 (0.0 ~ 1.0).
    confidence_list = result.boxes.conf.cpu().numpy()
    # 레이블 인덱스 목록: 감지된 객체의 클래스 인덱스.
    label_index_list = result.boxes.cls.cpu().numpy()

    for bounding_box, confidence, label_index in zip(bounding_box_list, confidence_list, label_index_list) :
        color = random_color()

        x1, y1, x2, y2 = bounding_box
        label_text = label_list[label_index]

        draw.rectangle([(x1, y1), (x2, y2)], outline=color, width=2)
        draw.text((x1, y1), text=f"{label_text} : {(confidence*100):.2f}%", fill=color)
    image = cv2.cvtColor(image_arr, cv2.COLOR_BGR2RGB)
    return image


with gr.Blocks() as demo :

    def stream_webcam(image_arr) : 
        image = detect_objects(image_arr)
        return image
    
    with gr.Row() :
        webcam_image = gr.Image(label="실시간 화면", sources="webcam", streaming=True)
        output_image = gr.Image(label="검출 화면", type="pil", interactive=False)

    webcam_image.stream(fn=stream_webcam, inputs=[webcam_image], outputs=[output_image])

demo.launch()

    

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.



0: 384x640 2 persons, 3810.4ms
Speed: 92.7ms preprocess, 3810.4ms inference, 129.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1958.5ms
Speed: 14.3ms preprocess, 1958.5ms inference, 1291.3ms postprocess per image at shape (1, 3, 384, 640)
